In [1077]:
!pip install polars 
!pip install pandas

import polars as pl
import json
import os
import datetime
import pandas as pd

In [1078]:
  #---------------------------------------------------------------------------------------------------------------

target_score = {}

Weight_Vector = [0,1,1]

iterating_time = 1  

Min_Length_Per_Day = datetime.timedelta(hours=1)

Local_wdt_start = datetime.time(9, 0, 0)
Local_wdt_end = datetime.time(17, 0, 0)


In [1079]:
#WE WOULD NEED A JSON_FILE AS AN INPUT
'''
json_file='/Users/ankithrangan/Desktop/ChallengeData/seen_files/input_2.json'


#selecting attendees from json file
with open(json_file,'r') as file:
    data=json.load(file)
attendees=data['attendees']



#example attendees
'''


attendees={
    'Mumbai': 0,
    'London': 1,
    'Paris': 20,
    'Hong Kong': 0,
    'Singapore': 0,
    'Shanghai': 0,
    'Dubai': 0,
    'Zürich': 0,
    'Geneva': 0,
    'Aarhus': 0,
    'Sydney': 0,
    'Wroclaw': 0,
    'Budapest': 0
}

start_date = datetime.datetime(2023,5,10,10,0,0)
end_date = datetime.datetime(2023,5,15,10,0,0)

Event_Length = datetime.timedelta(hours = 2)


In [1080]:
# Read the CSV file
'''
a = 5

# Open and read the JSON file
def file_list(json_file):
    # Read the JSON file
    with open(json_file, "r") as file:
        data = json.load(file)
    start_date_str = data["availability_window"]["start"]
    end_date_str = data["availability_window"]["end"]

    start_date = datetime.datetime.strptime(start_date_str.replace("Z", ""), "%Y-%m-%dT%H:%M:%S")
    end_date = datetime.datetime.strptime(end_date_str.replace("Z", ""), "%Y-%m-%dT%H:%M:%S")

    files = []

    # Loop from start to end date within availability window
    current_date = start_date - datetime.timedelta(days=a)
    while current_date.date() <= end_date.date():
        # For each file, construct path like ChallengeData/schedules/2025/12/10.csv
        file_path = os.path.join(
            "schedules",
            str(current_date.year),
            f"{current_date.month:02d}",
            f"{current_date.day:02d}.csv"
        )
        files.append(file_path)
        current_date += datetime.timedelta(days=1)

    return files

# example path: /Users/ankithrangan/Desktop/ChallengeData/schedules/2023/05/04.csv
#/Users/ankithrangan/Desktop/ChallengeData/schedules/2025/03/03.csv
schedule_files=[file_list(json_file)] #list of csv files
new_files = schedule_files[0]
new_files
'''

'\na = 5\n\n# Open and read the JSON file\ndef file_list(json_file):\n    # Read the JSON file\n    with open(json_file, "r") as file:\n        data = json.load(file)\n    start_date_str = data["availability_window"]["start"]\n    end_date_str = data["availability_window"]["end"]\n\n    start_date = datetime.datetime.strptime(start_date_str.replace("Z", ""), "%Y-%m-%dT%H:%M:%S")\n    end_date = datetime.datetime.strptime(end_date_str.replace("Z", ""), "%Y-%m-%dT%H:%M:%S")\n\n    files = []\n\n    # Loop from start to end date within availability window\n    current_date = start_date - datetime.timedelta(days=a)\n    while current_date.date() <= end_date.date():\n        # For each file, construct path like ChallengeData/schedules/2025/12/10.csv\n        file_path = os.path.join(\n            "schedules",\n            str(current_date.year),\n            f"{current_date.month:02d}",\n            f"{current_date.day:02d}.csv"\n        )\n        files.append(file_path)\n        current_d

In [1081]:
'''
for i in range(len(new_files)):
    new_files[i] = f"/Users/ankithrangan/Desktop/ChallengeData/{new_files[i]}"
new_files
'''

'\nfor i in range(len(new_files)):\n    new_files[i] = f"/Users/ankithrangan/Desktop/ChallengeData/{new_files[i]}"\nnew_files\n'

In [1082]:
#############################################
""" #############################################
from pathlib import Path
# If your file_list() returned relative paths like "schedules/2025/12/05.csv"
base = Path("/Users/ankithrangan/Desktop/ChallengeData")

# If schedule_files was [[...]], flatten it first:
flat_files = schedule_files[0] if isinstance(schedule_files[0], list) else schedule_files

# Build absolute paths and check existence
abs_paths = [base / Path(p.strip()) for p in flat_files]
existing = [str(p) for p in abs_paths if p.is_file()]
missing  = [str(p) for p in abs_paths if not p.is_file()]

if missing:
    print(f"Missing {len(missing)} files. First few:\n", "\n".join(missing[:5]))

if not existing:
    raise FileNotFoundError("No existing CSVs to read after filtering.")

# Combine them into a single DataFrame
schedules = pl.scan_csv(existing).collect()

"""#############################################

'''
schedules=pl.scan_csv(new_files, infer_schema_length=10000).collect() #list of paths to access csv files (within availability)
schedules
#/Users/ankithrangan/Desktop/ChallengeData/schedules/2023/05/04.csv
'''

'\nschedules=pl.scan_csv(new_files, infer_schema_length=10000).collect() #list of paths to access csv files (within availability)\nschedules\n#/Users/ankithrangan/Desktop/ChallengeData/schedules/2023/05/04.csv\n'

In [1083]:

df = pl.read_csv("12.csv", ignore_errors=True)



df = df.with_columns([
    pl.col("SCHEDULED_DEPARTURE_DATE_TIME_UTC").str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S%.3f", strict=False).alias("UTC_DEP"),
    pl.col("SCHEDULED_DEPARTURE_DATE_TIME_LOCAL").str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S%.3f", strict=False).alias("LCL_DEP"),
    pl.col("SCHEDULED_ARRIVAL_DATE_TIME_UTC").str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S%.3f", strict=False).alias("UTC_ARR"),
    pl.col("SCHEDULED_ARRIVAL_DATE_TIME_LOCAL").str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S%.3f", strict=False).alias("LCL_ARR")
])


df = df[:,["DEPAPT",       #1
           "DEPCITY",      #2
           "ARRAPT",       #3
           "ARRCITY",      #4
           "UTC_DEP",      #5
           "LCL_DEP",      #6
           "UTC_ARR",      #7
           "LCL_ARR",      #8
           "TOTAL_SEATS",  #9
           "OAG_SCHEDULE_FINGERPRINT" #10
          ]]

df = df.with_columns([(pl.col("UTC_ARR") - pl.col("LCL_ARR")).alias("LCL2UTC")])
df = df.with_columns([(pl.col("UTC_ARR") - pl.col("UTC_DEP")).alias("duration")])

df = df[:,["DEPAPT",       #1
           "DEPCITY",      #2
           "ARRAPT",       #3
           "ARRCITY",      #4
           "UTC_DEP",      #5
           "LCL_DEP",      #6
           "UTC_ARR",      #7
           "LCL_ARR",      #8
           "LCL2UTC",
           "duration",
           "TOTAL_SEATS",  #9
           "OAG_SCHEDULE_FINGERPRINT" #10
          ]]


print(df)







shape: (156_507, 12)
┌────────┬─────────┬────────┬─────────┬───┬──────────────┬─────────────┬─────────────┬─────────────┐
│ DEPAPT ┆ DEPCITY ┆ ARRAPT ┆ ARRCITY ┆ … ┆ LCL2UTC      ┆ duration    ┆ TOTAL_SEATS ┆ OAG_SCHEDUL │
│ ---    ┆ ---     ┆ ---    ┆ ---     ┆   ┆ ---          ┆ ---         ┆ ---         ┆ E_FINGERPRI │
│ str    ┆ str     ┆ str    ┆ str     ┆   ┆ duration[ms] ┆ duration[ms ┆ i64         ┆ NT          │
│        ┆         ┆        ┆         ┆   ┆              ┆ ]           ┆             ┆ ---         │
│        ┆         ┆        ┆         ┆   ┆              ┆             ┆             ┆ str         │
╞════════╪═════════╪════════╪═════════╪═══╪══════════════╪═════════════╪═════════════╪═════════════╡
│ VVI    ┆ SRZ     ┆ TJA    ┆ TJA     ┆ … ┆ 4h           ┆ 1h          ┆ 168         ┆ e0bc91c9ecd │
│        ┆         ┆        ┆         ┆   ┆              ┆             ┆             ┆ 88ca60117c1 │
│        ┆         ┆        ┆         ┆   ┆              ┆            

In [1084]:

'''
airport_dictionary = {
    'Mumbai': ['BOM'],
    'London': ['LHR', 'STN', 'LGW', 'LTN'],
    'Paris': ['CDG', 'BVA', 'ORY'],
    'Hong Kong': ['HKG'],
    'Singapore': ['SIN'],
    'Shanghai': ['PVG', 'SHA'],
    'Dubai': ['DXB'],
    'Zürich': ['ZRH'],
    'Geneva': ['GVA'],
    'Aarhus': ['AAR'],
    'Sydney': ['SYD'],
    'Wroclaw': ['WRO'],
    'Budapest': ['BUD']
}
'''
city_dictionary = {
    'Mumbai': ['BOM'],
    'London': ['LON'],
    'Paris': ['PAR'],
    'Hong Kong': ['HKG'],
    'Singapore': ['SIN'],
    'Shanghai': ['SHA'],
    'Dubai': ['DXB'],
    'Zürich': ['ZRH'],
    'Geneva': ['GVA'],
    'Aarhus': ['AAR'],
    'Sydney': ['SYD'],
    'Wroclaw': ['WRO'],
    'Budapest': ['BUD']
}


#adds corresponding country code to the 'depparture codes'
dep_codes = []
for city in attendees.keys():
    for i in city_dictionary[city] :
        for j in range(0, attendees[city]) :
            dep_codes.append(i)
print(dep_codes)

'''
#filter relevant airports (ie in dep codes)
filter_expression = (
    (pl.col("DEPCITY").is_in(dep_codes)) | 
    (pl.col("ARRCITY").is_in(dep_codes))
)
'''#taken out because there is a better way
'''
filtered_schedules = schedules.filter(filter_expression) #gives all rows with airports mentioned in the input
filtered_schedules
'''


['LON', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR']


'\nfiltered_schedules = schedules.filter(filter_expression) #gives all rows with airports mentioned in the input\nfiltered_schedules\n'

In [1085]:
print(dep_codes)

['LON', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR']


In [1086]:
df = df.filter(pl.col("DEPCITY").is_in(dep_codes))
df = df.filter(pl.col("UTC_ARR") < end_date )
print(df)

shape: (7_423, 12)
┌────────┬─────────┬────────┬─────────┬───┬──────────────┬─────────────┬─────────────┬─────────────┐
│ DEPAPT ┆ DEPCITY ┆ ARRAPT ┆ ARRCITY ┆ … ┆ LCL2UTC      ┆ duration    ┆ TOTAL_SEATS ┆ OAG_SCHEDUL │
│ ---    ┆ ---     ┆ ---    ┆ ---     ┆   ┆ ---          ┆ ---         ┆ ---         ┆ E_FINGERPRI │
│ str    ┆ str     ┆ str    ┆ str     ┆   ┆ duration[ms] ┆ duration[ms ┆ i64         ┆ NT          │
│        ┆         ┆        ┆         ┆   ┆              ┆ ]           ┆             ┆ ---         │
│        ┆         ┆        ┆         ┆   ┆              ┆             ┆             ┆ str         │
╞════════╪═════════╪════════╪═════════╪═══╪══════════════╪═════════════╪═════════════╪═════════════╡
│ LHR    ┆ LON     ┆ NWI    ┆ NWI     ┆ … ┆ -1h          ┆ 1d 5h       ┆ null        ┆ 20138f803d0 │
│        ┆         ┆        ┆         ┆   ┆              ┆             ┆             ┆ 21d81775760 │
│        ┆         ┆        ┆         ┆   ┆              ┆             ┆

In [1087]:

targets = []
target = df[:, ["ARRCITY"]]
target = target.with_columns([(pl.col("ARRCITY").count().over("ARRCITY") ).alias("frequency")])
target = target.filter(pl.col("frequency")>0).unique()
target = target.sort("frequency", descending = True)
target = target[:,["ARRCITY"]].rows()
for i in dep_codes :
    targets.append(i)
for i in target :
    if i[0] not in targets :
        targets.append(i[0])

target_apts = []
T_APT = df[:, ["ARRCITY","ARRAPT"]]
T_APT = T_APT.filter(pl.col("ARRCITY").is_in(targets)).unique()
T_APT = T_APT.with_columns([(pl.col("ARRCITY").count().over("ARRCITY") ).alias("freq")])
F_T_APT = T_APT
#################################### THIS SECTION WILL BE NEEDED FOR CONSIDERING ALL AIPORTS AT A MEETING CITY #########################
T_APT = T_APT[:,["ARRAPT"]].rows()
for i in T_APT :
    target_apts.append(i[0])
print(targets)
#print(target_apts)






['LON', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'PAR', 'XRE', 'NYC', 'LIL', 'DUB', 'BRS', 'AMS', 'XWS', 'LYS', 'BRU', 'MAD', 'LME', 'LIS', 'EDI', 'GVA', 'NTE', 'NCE', 'BOD', 'MRS', 'GLA', 'NCL', 'FRA', 'RNS', 'ANE', 'LAX', 'BHX', 'ETZ', 'SXB', 'RHE', 'CPH', 'BCN', 'MAN', 'TLS', 'ZRH', 'CHI', 'EXT', 'TUF', 'QQX', 'MIL', 'TTY', 'ROM', 'OSL', 'HAM', 'STO', 'IST', 'PLH', 'PRG', 'ATH', 'MUC', 'SFO', 'VAF', 'BFS', 'SIN', 'QQY', 'DUS', 'OXF', 'BOH', 'VIE', 'MPL', 'PIS', 'YTO', 'BER', 'AVN', 'DXB', 'DFW', 'GLO', 'WAS', 'CWL', 'YMQ', 'OPO', 'VCE', 'LBA', 'HEL', 'BUH', 'WAW', 'DOH', 'GOT', 'XQL', 'XNE', 'BOS', 'SOU', 'AGP', 'BSH', 'BLQ', 'BUD', 'QXB', 'ATL', 'PMI', 'PZE', 'USX', 'MIA', 'PCW', 'STR', 'CMN', 'XVC', 'FLR', 'SNN', 'ORK', 'BES', 'HOU', 'HAJ', 'TLV', 'ABZ', 'ALG', 'EAP', 'FNI', 'RAK', 'ZAG', 'PHL', 'FAO', 'BIO', 'SEA', 'NAP', 'BIQ', 'YVR', 'KYN', 'YYC', 'DEN', 'ORL', 'NWI', 'CAI', 'SAO', 'LCA'

In [1088]:
df_emissions = pl.read_csv("emissions.csv")

df_emissions = df_emissions.filter(pl.col("DEPARTURE_AIRPORT").is_in(dep_codes))####need codes in airports too
df_emissions = df_emissions.drop_nulls("ESTIMATED_CO2_TOTAL_TONNES")
df_emissions = df_emissions.filter(pl.col("SEATS")!= 0)
df_emissions = df_emissions.sort("SEATS")
df_emissions = df_emissions.with_columns([(pl.col("ESTIMATED_CO2_TOTAL_TONNES") / pl.col("SEATS")).alias("EMISSIONSpSEAT")])

df_emissions = df_emissions[:,["DEPARTURE_AIRPORT",
                               "ARRIVAL_AIRPORT",
                               "EMISSIONSpSEAT",
                              # "SEATS"
                              ]]
df_emissions = df_emissions.with_columns([pl.col("EMISSIONSpSEAT").mean().over("DEPARTURE_AIRPORT","ARRIVAL_AIRPORT").alias("AV_EMI_SEAT")])
df_emissions = df_emissions[:,["DEPARTURE_AIRPORT","ARRIVAL_AIRPORT","AV_EMI_SEAT"]].unique()
df_emissions = df_emissions.filter(pl.col("ARRIVAL_AIRPORT").is_in(target_apts)) 

print(df_emissions)


shape: (0, 3)
┌───────────────────┬─────────────────┬─────────────┐
│ DEPARTURE_AIRPORT ┆ ARRIVAL_AIRPORT ┆ AV_EMI_SEAT │
│ ---               ┆ ---             ┆ ---         │
│ str               ┆ str             ┆ f64         │
╞═══════════════════╪═════════════════╪═════════════╡
└───────────────────┴─────────────────┴─────────────┘


In [1089]:
dft = df[:,["ARRCITY","LCL2UTC"]].unique()
print(dft)

shape: (378, 2)
┌─────────┬──────────────┐
│ ARRCITY ┆ LCL2UTC      │
│ ---     ┆ ---          │
│ str     ┆ duration[ms] │
╞═════════╪══════════════╡
│ LPA     ┆ -1h          │
│ BAH     ┆ -3h          │
│ CFE     ┆ -2h          │
│ RNW     ┆ -1h          │
│ AUH     ┆ -4h          │
│ …       ┆ …            │
│ BRS     ┆ -1h          │
│ BOS     ┆ 4h           │
│ BJA     ┆ -1h          │
│ SUF     ┆ -2h          │
│ CZL     ┆ -1h          │
└─────────┴──────────────┘


In [1090]:

def Make_DT(T, DT) :
    return datetime.datetime.combine(datetime.date(DT), T)
    
def End(start_time) :
    nightlength = (Make_DT(UTC_wdt_start, start_time) - Make_DT(UTC_wdt_end, start_time)) + datetime.timedelta(days=1)
    daylength = Make_DT(UTC_wdt_end, start_time) - Make_DT(UTC_wdt_start, start_time)
    nights = (Event_Length//daylength)
    return start_time + Event_Length + (nightlength * nights)
'''
def get_Diff(target) :
    dft2 = dft.filter(pl.col("ARRCITY") == target)
    return dft2[0,1]
'''

'\ndef get_Diff(target) :\n    dft2 = dft.filter(pl.col("ARRCITY") == target)\n    return dft2[0,1]\n'

In [1091]:
print(df)

shape: (7_423, 12)
┌────────┬─────────┬────────┬─────────┬───┬──────────────┬─────────────┬─────────────┬─────────────┐
│ DEPAPT ┆ DEPCITY ┆ ARRAPT ┆ ARRCITY ┆ … ┆ LCL2UTC      ┆ duration    ┆ TOTAL_SEATS ┆ OAG_SCHEDUL │
│ ---    ┆ ---     ┆ ---    ┆ ---     ┆   ┆ ---          ┆ ---         ┆ ---         ┆ E_FINGERPRI │
│ str    ┆ str     ┆ str    ┆ str     ┆   ┆ duration[ms] ┆ duration[ms ┆ i64         ┆ NT          │
│        ┆         ┆        ┆         ┆   ┆              ┆ ]           ┆             ┆ ---         │
│        ┆         ┆        ┆         ┆   ┆              ┆             ┆             ┆ str         │
╞════════╪═════════╪════════╪═════════╪═══╪══════════════╪═════════════╪═════════════╪═════════════╡
│ LHR    ┆ LON     ┆ NWI    ┆ NWI     ┆ … ┆ -1h          ┆ 1d 5h       ┆ null        ┆ 20138f803d0 │
│        ┆         ┆        ┆         ┆   ┆              ┆             ┆             ┆ 21d81775760 │
│        ┆         ┆        ┆         ┆   ┆              ┆             ┆

In [1092]:
score_L = []
for target in targets :
    times = []
    emissions = []
    skip = False
    
    
    for depcity in dep_codes :
        df3 = df.filter(pl.col("ARRCITY") == target)
        df3 = df3.filter(pl.col("DEPCITY") == depcity)
        df3 = df3.with_columns([pl.col("duration").mean().alias("avv")])
        df3 = df3.sort("UTC_ARR", descending = True)
        df3 = df3[0,["duration"]]
        if df3.is_empty() :
            skip = True
            break
        else :
            if depcity == target :
                times.append(datetime.timedelta(hours = 0).total_seconds())
            else :
                times.append(df3.item(0,0).total_seconds())
    if skip == True :
        continue
    T_time = 0
    for i in times :
        T_time += i
    avvv = T_time / len(times)
    for i in times :
        avsq = (avvv - i )**2
    T_SDEV = (avsq/len(times))**0.5
    score = (T_time * Weight_Vector[1]) + (T_SDEV * Weight_Vector[2]) #'''T_emission * Weight_Vector[0] + '''
    score_L.append({"Target_City" : target,
                   "Score" : score,
                   "Fairness" : T_SDEV * Weight_Vector[2],
                    "Total_Time" : T_time * Weight_Vector[1]
                   })

    
score_DT = pl.DataFrame(score_L)
score_DT = score_DT.sort("Score")
print(score_DT.head(10))
'''
    T_emission = sum(emissions)
    A_time = sum(times)/(len(times))
    fairness = np.std(times)

    score = T_emission * Weight_Vector[0] + A_time * Weight_Vector[1] + fairness * Weight_Vector[2]

'''

shape: (10, 4)
┌─────────────┬──────────────┬────────────┬────────────┐
│ Target_City ┆ Score        ┆ Fairness   ┆ Total_Time │
│ ---         ┆ ---          ┆ ---        ┆ ---        │
│ str         ┆ f64          ┆ f64        ┆ f64        │
╞═════════════╪══════════════╪════════════╪════════════╡
│ PAR         ┆ 47286.314155 ┆ 486.314155 ┆ 46800.0    │
│ PAR         ┆ 47286.314155 ┆ 486.314155 ┆ 46800.0    │
│ PAR         ┆ 47286.314155 ┆ 486.314155 ┆ 46800.0    │
│ PAR         ┆ 47286.314155 ┆ 486.314155 ┆ 46800.0    │
│ PAR         ┆ 47286.314155 ┆ 486.314155 ┆ 46800.0    │
│ PAR         ┆ 47286.314155 ┆ 486.314155 ┆ 46800.0    │
│ PAR         ┆ 47286.314155 ┆ 486.314155 ┆ 46800.0    │
│ PAR         ┆ 47286.314155 ┆ 486.314155 ┆ 46800.0    │
│ PAR         ┆ 47286.314155 ┆ 486.314155 ┆ 46800.0    │
│ PAR         ┆ 47286.314155 ┆ 486.314155 ┆ 46800.0    │
└─────────────┴──────────────┴────────────┴────────────┘


'\n    T_emission = sum(emissions)\n    A_time = sum(times)/(len(times))\n    fairness = np.std(times)\n\n    score = T_emission * Weight_Vector[0] + A_time * Weight_Vector[1] + fairness * Weight_Vector[2]\n\n'